# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline
import pandas as pd
import warnings
import scipy.stats as stats
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('homepage_actions.csv')

In [3]:
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [5]:
counts = df.nunique()
counts

timestamp    8188
id           6328
group           2
action          2
dtype: int64

In [6]:
df.groupby(['group','action']).count()

timestamp    id
group      action                 
control    click         932   932
           view         3332  3332
experiment click         928   928
           view         2996  2996

In [7]:
# Check for duplicates between control and experimental
ids_of_experimental_group = df[df.group=='experiment']['id'].tolist()
ids_of_control_group = df[df.group=='control']['id'].tolist()
set(ids_of_experimental_group) & set(ids_of_control_group)

set()

In [8]:
# Check for duplicates between click but didn't view
ids_of_clicks_group = set(df[df.action=='click']['id'])
ids_of_view_group = set(df[df.action=='view']['id'])

ids_of_clicks_group.issubset(ids_of_view_group)

True

Because all clicker IDs are a subset of all viewer IDs, then there is no one who clicked but didn't view.

### Questions & answers about dataset

* Q: How many unique users? 6,328 unique users based on ID. 4,264 in the control group and 3,924 in the experimental group.
* Q: How many viewers also clicked? A: 1,860
* Q: Are there any anomalies with the data; did anyone click who didn't view? A: No
* Q: Is there any overlap between the control and experiment groups? A: No
* Q: If so, how do you plan to account for this in your experimental design? A: N/A

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [10]:
# My initial instinct here was to use a chi-squared test, but I think it lead me to the wrong answer. I'm commenting the code out.
# I relied on the lab solution after that.

# df['count'] = 1
# df['clicker'] = np.where(df['action'] == 'click', 1, 0)
# df['experimental'] = np.where(df['group'] == 'experiment', 1, 0)

# contingency_table = pd.crosstab(index=df["clicker"], columns=df["experimental"])

# chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# print("chi-squared p-value:", p)

# results_table = pd.concat([pd.DataFrame(expected), contingency_table])
# results_table.columns.name = "experimental"

# results_table.index = ["0 (expected)", "1 (expected)", "0 (actual)", "1 (actual)"]
# results_table.index.name = "clicker"
# results_table

In [11]:
# """
# Based on the chi-squared test, we'd expect 891 members of the experimental group to click, and actually 928 of them clicked.
# While that is good, the p-value is just 0.06, which is larger than a standard alpha of 0.05.
# Given that, I'd say we failed to reject the null hypothesis.
# """

In [12]:
# The following is based on me looking at the lab solution.

df['count'] = 1
control = df[df.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)



print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head()

def welch_t(a, b):
    
    """ Calculate Welch's t-statistic for two samples. """

    numerator = a.mean() - b.mean()
    
    # “ddof = Delta Degrees of Freedom”: the divisor used in the calculation is N - ddof, 
    #  where N represents the number of elements. By default ddof is zero.
    
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    
    return np.abs(numerator/denominator)

def welch_df(a, b):
    
    """ Calculate the effective degrees of freedom for two samples. """
    
    s1 = a.var(ddof=1) 
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator

def p_value(a, b, two_sided=False):

    t = welch_t(a, b)
    df = welch_df(a, b)
    
    p = 1-stats.t.cdf(np.abs(t), df)
    
    if two_sided:
        return 2*p
    else:
        return p
    
p_value(control.click, experiment.click)

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [13]:
#Your code here
control_ctr = (control['click'].sum()) / control['view'].sum()
expected_ex_clicks = experiment['view'].sum() * control_ctr

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [17]:
#Your code here
import math
n = len(experiment)
p = control_ctr
var = abs(n*p*(1-p))
std = np.sqrt(var)
std_diff = (experiment['click'].sum() - expected_ex_clicks) / std
std_diff

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [18]:
#Your code here
import scipy.stats as stats
p_val = stats.norm.sf(std_diff)
p_val

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **The p-value here is lower than the previous statistical test. However, both are below an alpha of 0.05. Therefore I'd say we reject the null hypotheses. Users who visit the experimental version of the homepage appear more likely to click the button.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.